In [1]:
import os, shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Understanding

In [2]:
# Specify path to the dataset
train_dir = '/kaggle/input/shopee-product-detection-open/train/train/train/'
test_dir = '/kaggle/input/shopee-product-detection-open/test/test/'

In [3]:
# Function to count number of images
def count_images(location):
    counter = 0
    for path, subdirs, files in os.walk(location):
        for name in files:
            if name.endswith(".jpg"):
                counter = counter + 1
    
    return(counter)

In [4]:
# Count number of images
total_train = count_images(train_dir)
total_test = count_images(test_dir)

print("Total training images:", total_train)
print("Total test images:", total_test)

Total training images: 105390
Total test images: 12192


# Data Preparation

In [5]:
# Specify hyperparameters
batch_size = 64
epochs = 2
IMG_HEIGHT = 299
IMG_WIDTH = 299
learning_rate = 0.0001

In [6]:
# Define image generator
train_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

In [7]:
# Define flow from the training data
train_generator = train_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

Found 105390 images belonging to 42 classes.


In [8]:
# Define flow from the test data
test_generator = test_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=test_dir,
    shuffle=False,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

Found 12192 images belonging to 1 classes.


# Modeling

In [9]:
# Create the base model from the pre-trained model InceptionResNetV2
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
base_model = tf.keras.applications.InceptionResNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

219062272/219055592 [==============================] - 3s 0us/step


In [10]:
# Show the number of layers in the base model
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  780


In [11]:
# Add a classification head
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(42)
])

In [12]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [13]:
# Show summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 8, 8, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 42)                64554     
Total params: 54,401,290
Trainable params: 54,340,746
Non-trainable params: 60,544
_________________________________________________________________


In [14]:
# Create a callback that saves the model
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model.h5',
    save_best_only=True,
    verbose=1
)

In [15]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=total_train//batch_size,
    epochs=epochs,
    callbacks=[callback]
)

Epoch 1/2
1646/1646 [==============================] - 3175s 2s/step - loss: 0.9469 - accuracy: 0.7424
Epoch 2/2
1646/1646 [==============================] - 2860s 2s/step - loss: 0.5393 - accuracy: 0.8468


In [16]:
# Save the model
model.save('model.h5')

# Evaluation

In [17]:
# Load the best model
model = tf.keras.models.load_model('model.h5')

In [18]:
# Make prediction for test data
predictions = np.argmax(model.predict(test_generator), axis=-1)

In [19]:
# Get filenames of test data
filenames = test_generator.filenames

In [20]:
# Get list of test images for submission
test = pd.read_csv('/kaggle/input/shopee-product-detection-open/test.csv')

In [21]:
# Generate submission file
submission=pd.DataFrame({"filename": filenames, "category": predictions})
submission.category = submission.category.astype('str')
submission['filename'] = submission.apply(lambda row: row[0][5:], axis=1)
submission['category'] = submission.apply(lambda row: row[1].zfill(2), axis=1)
submission = pd.merge(test[['filename']], submission, on='filename', how='left')
submission.to_csv('submission.csv', index=False, header=True)